In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import soundfile as sf
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_io as tfio
from tensorboard import notebook

import keras.models
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten
from keras.layers import Dropout
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler


from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, Lambda
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16

In [2]:
def create_model(num_labels):
    info_input = Input((2,))
    info = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info_input)
    info = Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dropout(.33)(info)
    info = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dropout(.33)(info)
    info = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001))(info)
    info = Dropout(.33)(info)
    output = Dense(units = num_labels, activation ='softmax')(info)
    
    model = Model(info_input, [output])
    model.summary()
    optimizer = keras.optimizers.Adam(learning_rate=0.00025, decay=0.0025)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [3]:
def train_model(trainAttrX, trainY, testAttrX, testY, num_labels, batch_amt, epoch_amt, save):
    model = create_model(num_labels)

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    model.fit( x=[trainAttrX], y=trainY, validation_data=([testAttrX], testY),
        epochs=epoch_amt, batch_size=batch_amt, callbacks=[tensorboard_callback], verbose=False)
    
    if save:
        model.save("model")

    return model

In [4]:
def load_attributes(inputPath):
    cols = ["latitude", "longitude", "filename", "primary_label"]
    df = pd.read_csv(inputPath, skipinitialspace=True, usecols=cols)
    df = shuffle(df)
    df.reset_index(inplace=True, drop=True)
    return df

In [5]:
def process_attributes(inputPath, train, test):
    continuous = ["latitude", "longitude"]
    
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[continuous])
    testX = cs.transform(test[continuous])
    
    return (trainX, testX)

In [6]:
df = load_attributes("./Data/train_metadata.csv")

In [7]:
split = train_test_split(df, test_size=0.2, random_state=0)
(trainAttrX, testAttrX) = split
label_encoder = LabelEncoder()
trainY = to_categorical(label_encoder.fit_transform(trainAttrX["primary_label"]))
testY = to_categorical(label_encoder.fit_transform(testAttrX["primary_label"]))

(trainAttrX, testAttrX) = process_attributes(df,
    trainAttrX, testAttrX)


In [8]:
num_labels = trainY.shape[1]

In [ ]:
train_model(trainAttrX, trainY, testAttrX, testY, num_labels, 128, 150, True)

#model = keras.models.load_model("model")

#preds = model.predict([testImagesX, testAttrX])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 16)                48        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0     

In [ ]:
prediction = predict_file("./Data/Audio/aldfly/XC477348.ogg", model, label_encoder,
                              42.0458, -79.441, "2019-05-27")
prediction